In [12]:
import gym
import random
import numpy as np
import tensorflow as tf
from collections import deque
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

In [20]:
class Policy(nn.Module):

  def __init__(self,state_size=4,hidden_size=128,num_actions=2):
    super(Policy,self).__init__()

    self.fc1 = nn.Linear(state_size,hidden_size)
    self.fc2 = nn.Linear(hidden_size,num_actions)

  def forward(self,x):

    fcHid = F.relu(self.fc1(x))
    out = self.fc2(fcHid)
    p = F.softmax(out,dim=1)

    return p

  def chooseAct(self,x):
    x = torch.from_numpy(x).float().unsqueeze(0)
    prob = self.forward(x)
    model = Categorical(prob)
    act = model.sample() # randomly sample an action

    return act.item(),model.log_prob(act)


In [21]:
def reinforce(policy, optimizer, num_episodes = 500, max_timesteps = 500, gamma = 0.99):
  scores = []
  scores_deque = deque(maxlen = 100)
  for episode in range(num_episodes):
    state = env.reset()
    rewards = []
    log_probs_all = []
    # simulate an episode until end
    for t in range(max_timesteps):
      action,log_probs = policy.chooseAct(state)
      state,reward,done,_ = env.step(action)
      rewards.append(reward)
      log_probs_all.append(log_probs)
      if done:
        break
    scores.append(sum(rewards))
    scores_deque.append(scores)

    # calculate discounted returns for the episode using rewards obtained
    discounted_returns = []
    sumR = 0
    for r in reversed(rewards):
      sumR = sumR + gamma * r #r + gamma*(sumR)
      discounted_returns.insert(0,sumR)

    # Calculate the loss
    policy_loss = []
    i = 0
    for log_prob in log_probs_all:
        R = discounted_returns[i]
        policy_loss.append(-log_prob * R)
        i += 1
    policy_loss = torch.cat(policy_loss).sum()

    optimizer.zero_grad()
    policy_loss.backward()
    optimizer.step()
    print(f"Episode {episode}: Reward = {sum(rewards)}")
  return scores


In [ ]:
n_episodes = 1000
num_runs = 5
scores_all_runs = np.zeros((num_runs,n_episodes))
for r in range(num_runs):
  env = gym.make('CartPole-v1')
  policy = Policy()
  optimizer = optim.Adam(policy.parameters(), lr=1e-3)
  scores = reinforce(policy, optimizer, num_episodes = n_episodes)
  scores_all_runs[r,:] = scores

In [27]:
# saving the scores file across 5 runs
import pandas as pd
file_path = '/content/REINFORCE_wout_baseline_cartpole.xlsx'

pd.DataFrame(scores_all_runs).to_excel(file_path,index=False)
